In [1]:
import pandas as pd
import numpy as np

import os

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.layers import LSTM, TimeDistributed, ConvLSTM2D

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,f1_score,roc_auc_score,recall_score,precision_score

C:\Users\ESA LAB\anaconda3\envs\HighHeel\lib\site-packages\pydot.py:17: UserWarning: `pydot` could not import `dot_parser`, so `pydot` will be unable to parse DOT files. The error was:  No module named 'pyparsing'
  warnings.warn(


In [2]:
data_path = os.getcwd() + "/dataset/total_concentrated.csv"
# features = ["A_F", "A_V", "A_L", "L_F", "L_T", "L_L", "T_F", "T_V", "T_L"]

dataset = pd.read_csv(data_path)
time = list(dataset['time'])
dataframe = dataset[["A_F", "A_V", "A_L", "L_F", "L_V", "L_L", "T_F", "T_V", "T_L"]].values
dataset = dataset[["A_F", "A_V", "A_L", "L_F", "L_V", "L_L", "T_F", "T_V", "T_L", "Action"]].values

window_length = int(1*64)
total_windows = int((len(dataset))/window_length)

In [3]:
dataframe = (dataframe-dataframe.mean())/dataframe.std()

In [4]:
'''
X = np.empty((total_windows, window_length, 9))
y = np.empty((total_windows, 1))
j = 0

window_count = 0
for items in range(total_windows):
    for i in range(window_length):
        if i == 0:
            y[j] = dataset[int(window_count*window_length), 9]
            j = j + 1
        for data in range(9):
            X[items, i, data] = dataset[int(window_count*window_length)+i, data]
    window_count = window_count+1

del window_count, i, j, items, data
'''

'\nX = np.empty((total_windows, window_length, 9))\ny = np.empty((total_windows, 1))\nj = 0\n\nwindow_count = 0\nfor items in range(total_windows):\n    for i in range(window_length):\n        if i == 0:\n            y[j] = dataset[int(window_count*window_length), 9]\n            j = j + 1\n        for data in range(9):\n            X[items, i, data] = dataset[int(window_count*window_length)+i, data]\n    window_count = window_count+1\n\ndel window_count, i, j, items, data\n'

In [5]:
indices = time
groups = []
group_count = 0
temp = []
lenOfGroup = []
length_count = 0

for i in range(len(indices)):
    if i == (len(indices) - 1):
        temp.append(indices[i])
        length_count = length_count + 1

        groups.append(temp)
        lenOfGroup.append(length_count)
        length_count = 0
        temp = []
        break
    temp.append(indices[i])
    length_count = length_count + 1
    if (indices[i+1] - 20 > indices[i]):
        group_count = group_count + 1

        lenOfGroup.append(length_count)
        length_count = 0

        groups.append(temp)
        temp = []

del temp, indices, group_count, length_count, i

確認每段資料能不能整除64/2, 並輸出不能整除的個數

In [6]:
countOfUndivisible = 0
total_windows_with_overlap = 0

start_index = 0

stop_Indexs = []
stop_Index = -window_length


for lengths in lenOfGroup:
    stop_Index = stop_Index + lengths
    stop_Indexs.append(stop_Index)
    total_windows_with_overlap = total_windows_with_overlap + int(lengths/window_length*2 -1)
    
    if lengths % (window_length/2) != 0:
        countOfUndivisible = countOfUndivisible + 1
        print(lengths)

print(countOfUndivisible)
del countOfUndivisible, lengths

0


In [7]:
X_ = np.empty((total_windows_with_overlap, window_length*9))
y_ = np.empty((total_windows_with_overlap, 1))

stop_Index = 0
window_count = 0

for window in range(total_windows_with_overlap):
    for i in range(window_length):
        if i == 0:
            y_[window] = dataset[int(window_count*window_length), 9]
        
        if int(window_count*window_length)<len(dataset)-window_length-1:
            for data in range(9):
               X_[window, i*9 + data] = dataframe[int(window_count*window_length) + i, data]
        if stop_Index < len(stop_Indexs):
            if int(window_count*window_length) == stop_Indexs[stop_Index]:
                window_count = window_count + 0.5
                stop_Index = stop_Index + 1
    window_count = window_count + 0.5

In [8]:
from sklearn.model_selection import StratifiedKFold

In [9]:
X, X_test, y, y_test = train_test_split(X_, y_, test_size=0.2, random_state=42)

In [10]:
unique, counts = np.unique(y, return_counts=True)

class_weight = {0:(1/counts[0])*len(y)/2, 1:(1/counts[1])*len(y)/2, 2:(1/counts[2])*len(y)/2}

In [11]:
skf = StratifiedKFold(n_splits=10, shuffle = True, random_state=42)
skf.get_n_splits(X, y)
print(skf)

StratifiedKFold(n_splits=10, random_state=42, shuffle=True)


In [12]:
def to_3Darray(array):
    arr_3d = np.empty((len(array), window_length, 9))


    arr_3d = np.reshape(array, (len(array), window_length, 9))
    return arr_3d

In [13]:
'''def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 0, 50, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]

    n_steps, n_length = 2, 32
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))

    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), 
                              input_shape=(None, n_length, n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)

    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy'''

"def evaluate_model(trainX, trainy, testX, testy):\n    verbose, epochs, batch_size = 0, 50, 64\n    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]\n\n    n_steps, n_length = 2, 32\n    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))\n    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))\n\n    model = Sequential()\n    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), \n                              input_shape=(None, n_length, n_features)))\n    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))\n    model.add(TimeDistributed(Dropout(0.5)))\n    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))\n    model.add(TimeDistributed(Flatten()))\n    model.add(LSTM(100))\n    model.add(Dropout(0.5))\n    model.add(Dense(100, activation='relu'))\n    model.add(Dense(n_outputs, activation='softmax'))\n    model.compile(loss='categorical_crossentr

In [14]:
'''def run_experiment(trainX, trainy, testX, testy, repeats):

    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    
    m, s = np.mean(scores), np.std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

    return m'''

"def run_experiment(trainX, trainy, testX, testy, repeats):\n\n    scores = list()\n    for r in range(repeats):\n        score = evaluate_model(trainX, trainy, testX, testy)\n        score = score * 100.0\n        print('>#%d: %.3f' % (r+1, score))\n        scores.append(score)\n    \n    m, s = np.mean(scores), np.std(scores)\n    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))\n\n    return m"

In [15]:
scores = list()

In [16]:
scores_in_fold = []
losses = []

In [17]:
scores_outside_fold = []

In [18]:
for i, (train_index, val_index) in enumerate(skf.split(X, y)):
    print("==> Fold #%d" % i)
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    X_train = to_3Darray(X_train)
    y_train = to_categorical(y_train)

    X_val = to_3Darray(X_val)
    y_val = to_categorical(y_val)


    
    verbose, epochs, batch_size = 0, 50, 64
    n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

    n_steps, n_length = 2, 32
    X_train = X_train.reshape((X_train.shape[0], n_steps, n_length, n_features))
    X_val = X_val.reshape((X_val.shape[0], n_steps, n_length, n_features))

    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), 
                              input_shape=(None, n_length, n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu')) #feature
    model.add(Dense(n_outputs, activation='softmax')) #試著輸出長度為100的向量(feature) 並絳維 看他的分布有無分開
    model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy']) #可能可以調weighting
    
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose, class_weight = class_weight)
    
    loss, accuracy = model.evaluate(X_val, y_val, batch_size=batch_size, verbose=0)
    

    y_pred = (model.predict(X_val) > 0.5).astype("int32")
    y_val = np.argmax(y_val, axis=1)
    y_pred = np.argmax(y_pred, axis=1)

    confus = confusion_matrix(y_val, y_pred, labels=None, sample_weight=None)
    tp = confus[1][1]
    tn = confus[0][0] + confus[0][2] + confus[2][0] + confus[2][2]
    fp = confus[1][0] + confus[1][2]
    fn = confus[0][1] + confus[2][1]

    precision = (tp/(tp + fp))*100
    recall =  (tp / (tp + fn))*100   #sensitivity
    F1_score = ((2*tp) / (2*tp + fp + fn))*100

    score = accuracy
    losses.append(loss)
    
    score = score * 100.0
    scores_in_fold.append(score)
    
    print('Loss: %.3f%% Accuracy: %.3f%% ' % (loss, score))
    print('precision: %.3f%% recall: %.3f%% F1 score: %.3f%%' % (precision, recall, F1_score))
    # print(confus)


    
    '''score = evaluate_model(X_train, y_train, X_val, y_val)
    score = score * 100.0
    print(score)
    scores.append(score)'''

    

==> Fold #0
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Loss: 0.279% Accuracy: 90.091% 
precision: 91.270% recall: 58.376% F1 score: 71.207%
==> Fold #1
Loss: 0.368% Accuracy: 83.753% 
precision: 88.889% recall: 44.711% F1 score: 59.495%
==> Fold #2
Loss: 0.414% Accuracy: 83.954% 
precision: 90.476% recall: 45.328% F1 score: 60.397%
==> Fold #3
Loss: 0.318% Accuracy: 88.274% 
precision: 81.746% recall: 54.068% F1 score: 65.087%
==> Fold #4
Loss: 0.374% Accuracy: 85.657% 
precision: 90.079% recall: 48.195% F1 score: 62.794%
==> Fold #5
Loss: 0.393% Accuracy: 84.902% 
precision: 94.841% recall: 46.863% F1 score: 62.730%
==> Fold #6
Loss: 0.355% Accuracy: 87.267% 
precision: 88.845% recall: 51.264% F1 score: 65.015%
==> Fold #7
Loss: 0.356% Accuracy: 86.059% 
precision: 87.251% recall: 49.324% F1 score: 63.022%
==> Fold #8
Loss: 0.284% Accuracy: 89.129% 
precision: 79.681% recall: 57.307% F1 scor

In [19]:
m, s = np.mean(scores_in_fold), np.std(scores_in_fold)
print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

Accuracy: 86.746% (+/-2.096)


In [20]:
X_test = to_3Darray(X_test)
#y_test = to_categorical(y_test)
X_test = X_test.reshape((X_test.shape[0], n_steps, n_length, n_features))


# loss, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)


In [21]:
y_test = to_categorical(y_test)
y_pred = (model.predict(X_test) > 0.5).astype("int32")
y_test = np.argmax(y_test, axis=1)
y_pred = np.argmax(y_pred, axis=1)

confus = confusion_matrix(y_test, y_pred, labels=None, sample_weight=None)

In [22]:
print(confus)

[[3894  408    7]
 [  76  530    5]
 [  15   31    3]]


In [23]:
accuracy_score(y_test, y_pred)*100

89.092372710807

In [24]:
accuracy = accuracy*100
print('Accuracy: %.3f%% ' % (score))

Accuracy: 88.374% 


In [25]:
from keras.models import load_model

In [26]:
model.save('my_model.h5')